# Play with merge

In [6]:
import obspy
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import ooipy
import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [7]:
url = 'https://rawdata-west.oceanobservatories.org/files/RS01SBPS/PC01A/08-HYDBBA103/2023/08/10/OO-HYVM2--YDH-2023-08-10T10:00:00.000000Z.mseed'
st = obspy.read(url)

In [9]:
len(st)

1201

In [70]:
# check if npts is consistent with no gaps in data
npts_total = 0
for k, tr in enumerate(st):
    npts_total += tr.stats.npts
if npts_total/64000/60 == 5:
    print('valid for gapless merge')

valid for gapless merge


In [117]:
# merge with no gap consideration (for fragmented hydrophone data)
data = []
for tr in st:
    data.append(tr.data)
data_cat = np.concatenate(data)

stats = dict(st[0].stats)
stats['starttime'] = obspy.UTCDateTime(url[-33:-6])
stats['endtime'] = obspy.UTCDateTime(starttime.datetime + datetime.timedelta(minutes=5))
stats['npts'] = len(data_cat)

obspy.Stream(traces = obspy.Trace(data_cat, header=stats))

1 Trace(s) in Stream:
OO.HYVM2..YDH | 2023-08-10T10:00:00.000000Z - 2023-08-10T10:04:59.999984Z | 64000.0 Hz, 19200000 samples

In [54]:
data_nm = data_cat[:64000*10] - np.mean(data_cat[:64000*10])
data_norm = data_nm / np.max(data_nm)

In [57]:
from scipy import io

In [58]:
io.wavfile.write('test_cat.wav', 64000, data_norm)

# test ooipy method

In [11]:
import ooipy
from datetime import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
starttime = datetime(2023, 8, 10, 10, 2)
endtime = datetime(2023, 8, 10, 10, 4)
node = 'PC01A'

hdata = ooipy.get_acoustic_data(starttime, endtime, node, verbose=True, mseed_file_limit=100, gapless_merge=True)

Fetching URLs...
Sorting valid URLs for Time Window...


100%|█████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]


gapless merge for 1201 Trace(s) in Stream:

OO.HYVM2..YDH | 2023-08-10T10:05:00.000016Z - 2023-08-10T10:05:00.085235Z | 64000.0 Hz, 5455 samples
...
(1199 other traces)
...
OO.HYVM2..YDH | 2023-08-10T10:05:00.000016Z - 2023-08-10T10:05:00.164766Z | 64000.0 Hz, 10545 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
gapless merge for 1201 Trace(s) in Stream:

OO.HYVM2..YDH | 2023-08-10T10:00:00.000016Z - 2023-08-10T10:00:00.085235Z | 64000.0 Hz, 5455 samples
...
(1199 other traces)
...
OO.HYVM2..YDH | 2023-08-10T10:00:00.000016Z - 2023-08-10T10:00:00.164766Z | 64000.0 Hz, 10545 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
gapless merge for 1201 Trace(s) in Stream:

OO.HYVM2..YDH | 2023-08-10T09:55:00.000016Z - 2023-08-10T09:55:00.085235Z | 64000.0 Hz, 5455 samples
...
(1199 other traces)
...
OO.HYVM2..YDH | 2023-08-10T09:55:00.000016Z - 2023-08-10T09:55:00.164766Z | 64000.0 Hz, 10545 samples

[Use "print(Stream.__str__(extended=True))

In [7]:
ooipy.__file__

'/home/jhrag/Code/ooipy/ooipy/__init__.py'